Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Imports:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_regression, SelectKBest

In [2]:
# Read in my data:
data = 'https://raw.githubusercontent.com/jmmiddour/Datasets/master/Renewable_Energy/UFO_sightings.csv'
ufos = pd.read_csv(data)

# Look at the first 5 rows:
ufos.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,Unnamed: 11
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,NaN
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,NaN
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,NaN
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,NaN
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,NaN


In [3]:
# Look at the values in Unnamed 11:
ufos['Unnamed: 11'].value_counts()

0.0    196
Name: Unnamed: 11, dtype: int64

In [4]:
# Since nothing of value to me in the Unnamed 11 column, I am dropping it:
ufos = ufos.drop('Unnamed: 11', axis=1)

# Check my work:
ufos.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [5]:
# Look at the data types of features:
ufos.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

In [6]:
# Look at the descriptive stats of all object columns:
ufos.describe(exclude='number')

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude
count,88875,88679,81356,76314,85757,88873,85772,88749,88875,88875
unique,76305,22018,68,5,29,730,9687,88348,513,19545
top,7/4/2010 22:00,seattle,ca,us,light,300,5 minutes,Fireball,12/12/2009,0
freq,36,570,10450,70293,17872,6457,4796,12,1624,1494


In [7]:
# Look at the length of a value that is the length it should be for formatting:
len(ufos['datetime'][23575])

16

In [8]:
# Create a new column with length values of datetime values:
ufos['dt_len'] = ufos['datetime'].str.len()
ufos.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,dt_len
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,16
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,16
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,16
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,16
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,16


In [9]:
# Look at the value counts of the new column:
ufos['dt_len'].value_counts()

15    42502
14    28229
16    11881
13     5001
22      675
21      388
23      198
20        1
Name: dt_len, dtype: int64

In [10]:
# Look at the number of observations in the dataframe:
ufos.shape[0]

88875

In [11]:
# Drop all rows with strange time (length more than 16):
ufos = ufos.drop(ufos.loc[ufos['dt_len'] >= 17].index)
ufos['dt_len'].value_counts()

15    42502
14    28229
16    11881
13     5001
Name: dt_len, dtype: int64

In [12]:
# Look at the observations in the dataframe again to check work:
ufos.shape[0]

87613

In [13]:
# Create a new column with length values of datetime values:
ufos['dtp_len'] = ufos['date posted'].str.len()
ufos.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,dt_len,dtp_len
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,16,9
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,16,10
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,16,9
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,16,9
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,16,9


In [14]:
# Look at the value counts of the new column:
ufos['dtp_len'].value_counts()

9      50319
10     18661
8      18479
138       11
134        7
       ...  
23         1
85         1
148        1
20         1
89         1
Name: dtp_len, Length: 90, dtype: int64

In [15]:
ufos[ufos['dtp_len'] > 10]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,dt_len,dtp_len
876,10/1/2006 12:00,NaN,NaN,NaN,NaN,0,NaN,NaN,((EDITORIAL COMMENT ABOUT THE UFO PHENOMEN)) ...,10/30/2006,0.0,15,64
1711,10/14/2004 13:00,NaN,NaN,NaN,NaN,0,NaN,NaN,With all the guns in this country...why hasn&#...,10/27/2004,0.0,16,77
1813,10/14/2011 22:30,NaN,nv,NaN,NaN,0,light,22,3 Green lights,10/19/2011,0.0,16,14
2856,10/17/2008 20:30,NaN,tx,NaN,NaN,0,oval,5 minutes,An oval shaped object in a photograph. ((NUF...,10/31/2008,0.0,16,94
3732,10/20/2013 18:30,NaN,ct,NaN,NaN,0,egg,2 hours,Bright light visible in W sky for long periods...,11/11/2013,0.0,16,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80501,8/9/2012 0:00,NaN,NaN,NaN,NaN,0,triangle,at least 2 hours,Multiple triangular shaped UFOs above the sout...,8/19/2012,0.0,13,68
82564,9/15/1974 21:00,NaN,ky,NaN,NaN,0,disk,?,craft had light under it and hovered over the ...,2/14/2008,0.0,15,93
86121,9/26/2007 0:00,NaN,NaN,NaN,NaN,0,NaN,NaN,This is not a Ufo report. It&#39s the only wa...,10/8/2007,0.0,14,134
87216,9/30/2012 23:00,NaN,NaN,NaN,NaN,0,NaN,NaN,I was checking out the reported UFO in the sky...,10/30/2012,0.0,15,144


In [16]:
# Show max rows:
pd.set_option('display.max_rows', None)

ufos['duration (hours/min)'].value_counts()

5 minutes                             4770
2 minutes                             3516
10 minutes                            3358
1 minute                              3097
3 minutes                             2534
30 seconds                            2334
15 minutes                            2091
10 seconds                            2005
5 seconds                             1831
20 minutes                            1455
30 minutes                            1343
1 hour                                1340
15 seconds                            1189
20 seconds                            1143
3 seconds                              940
4 minutes                              912
5 min                                  849
2 seconds                              693
2 hours                                654
10 min                                 651
2-3 minutes                            614
2 min                                  540
45 seconds                             524
unknown    

In [17]:
# Replace ? with unknown:
ufos['duration (hours/min)'] = ufos['duration (hours/min)'].replace('?', 'unknown')

# Look at value counts again:
ufos['duration (hours/min)'].value_counts()

5 minutes                             4770
2 minutes                             3516
10 minutes                            3358
1 minute                              3097
3 minutes                             2534
30 seconds                            2334
15 minutes                            2091
10 seconds                            2005
5 seconds                             1831
20 minutes                            1455
30 minutes                            1343
1 hour                                1340
15 seconds                            1189
20 seconds                            1143
3 seconds                              940
4 minutes                              912
5 min                                  849
2 seconds                              693
unknown                                689
2 hours                                654
10 min                                 651
2-3 minutes                            614
2 min                                  540
45 seconds 

In [18]:
# Replace #ERROR! with NaN and unsure with unknown:
ufos['duration (hours/min)'] = ufos['duration (hours/min)'].replace(['#ERROR!', 'unsure'], 
                                                                    [np.nan, 'unknown'])

# Look at value counts with less than 30:
print(ufos['duration (hours/min)'].value_counts() < 30)

5 minutes                             False
2 minutes                             False
10 minutes                            False
1 minute                              False
3 minutes                             False
30 seconds                            False
15 minutes                            False
10 seconds                            False
5 seconds                             False
20 minutes                            False
30 minutes                            False
1 hour                                False
15 seconds                            False
20 seconds                            False
3 seconds                             False
4 minutes                             False
5 min                                 False
unknown                               False
2 seconds                             False
2 hours                               False
10 min                                False
2-3 minutes                           False
2 min                           

In [14]:
# Need to change datetime and date posted to datetime:
ufos['datetime'] = pd.to_datetime(ufos['datetime'], infer_datetime_format=True)
ufos['date posted'] = pd.to_datetime(ufos['date posted'], infer_datetime_format=True)

# Check my work:
ufos.dtypes

ParserError: ignored